In [ ]:
###########################################################
#                                                         #
#               TRAITEMENT SIMUS FORTRAN                  #
#                     STATISTIQUES                        #
#                                                         #
#                         avec                            #
#                   - lecture arborescence                #
#                  analyse                                #
#                   - Delta Fluo                          #
#                   - Delta Energy GMol                   #
#                   - R/L Ca+                             #
#                   - distance inter ionique              #
#                   - trajectoire GMol                    #
#                                                         #
#                          02/2020     Adrien Poindron    #
###########################################################

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
matplotlib.rcParams['figure.figsize'] = [10.5,7]
matplotlib.rcParams.update({'font.size': 25})

# Imports

In [2]:
# %pylab

import numpy
import matplotlib
from matplotlib.patches import Ellipse
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

matplotlib.rcParams.update({'font.size': 21})
# matplotlib.rcParams["figure.dpi"] = 300

from IPython.display import display
from IPython.core.pylabtools import figsize, getfigs

from pylab import *
from numpy import *

from scipy.optimize import curve_fit

In [4]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

import matplotlib.patches as patches

from pathlib import Path
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [5]:
from natsort import natsorted   # natural sort marche de ouf
import re                       # find '/' in strings
import timeit
cm = pylab.get_cmap('summer')   # colormap par défaut color=cm(k/len(array))

In [69]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run "../../Functions/0.2/data_fct_Adrien_point_by_point.py"

In [7]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Sélection fichier

In [75]:
## GUI for data loading
# Select one data file all the way down to the directories
# Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG.dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

> Répertoire : /home/adrian/Documents/Programmes/Fortran_Jofre/20200609/DC03_RF19/Try00
> Filename : Temp_SimuType0_N01024_Vrf0055_Udc0.2000D+01V_D1.0_S1.0RFG.dat
> myslashpos | [0, 5, 12, 22, 33, 47, 56, 66]
> slashcond | 6
Points deleted because they were not complete []   0 pt(s)
Total number of data directories 5


In [76]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

['DC03_RF19', 'DC05_RF10', 'DC05_RF18', 'DC09_RF11', 'DC09_RF14']
[4]
['DC', []]
> condition names ['DC', 'RF']
> number of points 5
000 > DC03_RF19
001 > DC05_RF10
002 > DC05_RF18
003 > DC09_RF11
004 > DC09_RF14
> N_ions = 1024
> e_GMol = 50


In [77]:
## Data loading (1D arrays)
data_name, num_runs, PMandT, Gmol_data, r_LC_clip, dim_nu \
= data_retrieve(all_subdir, points_and_coord,condition_parameters,slash_cfg,mode='GMol') 

data0 = data_name[0]
data2 = data_name[1]
data4 = data_name[2]

PMvar = PMandT[0]
Tvar = PMandT[1]

deltaEc = Gmol_data[0]
deltaEcRel = Gmol_data[1]
t_c = Gmol_data[2]

> Points | 5
> Simulations pour chaque point | ['Try00']
Hello
['Temp_SimuType0_N01024_Vrf0055_Udc0.2000D+01V_D1.0_S1.0RFG.dat'
 'Temp_SimuType2_01_N01024_Vrf0055_Udc0.2000D+01V_D1.0_S1.0RFG50eV.dat'
 'Temp_SimuType4_01_N01024_Vrf0055_Udc0.2000D+01V_D1.0_S1.0RFG50eV.dat'
 'Temp_SimuType6_01_N01024_Vrf0055_Udc0.2000D+01V_D1.0_S1.0RFG50eV.dat']
0
Point n° 0
00 - 00  >  {}//home/adrian/Documents/Programmes/Fortran_Jofre/20200609/DC03_RF19/Try00
['Temp_SimuType0_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG.dat'
 'Temp_SimuType2_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV.dat'
 'Temp_SimuType4_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV.dat'
 'Temp_SimuType6_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG.dat'
 'Temp_SimuType6_01_N01024_Vrf0064_Udc0.3000D+01V_D1.0_S1.0RFG50eV.dat']
1
Point n° 1
01 - 00  >  {}//home/adrian/Documents/Programmes/Fortran_Jofre/20200609/DC05_RF10/Try00
['Temp_SimuType0_N01024_Vrf0071_Udc0.3000D+01V_D1.0_S1.0RFG.dat'
 'Temp_SimuType2_01_N01024_Vrf0071_Udc

In [ ]:
# Where SNR for eeach try in each  are compared to a threshold
# If SNR > some_threshold , we count it as a detection
# Then for each point the ratio number_of_detection/number_of_try is calculated
# This detection efficiency 

SNR_trig_threshold = 1
shapevar = ((len(points_and_coord), len(num_runs)))
trig_detection = np.zeros((shapevar))
for i in range(len(points_and_coord)):
    SNR = Gmol_data[2][i]
    for k,m in enumerate(SNR):
        if m > SNR_trig_threshold:
            trig_detection[i,k] = 1

print(mean(trig_detection,axis=1))

print(len(trig_detection))

# Where is written Detuning and how much it is
temp_pos = filename.find('0RFG')
Detuning = filename[temp_pos-7:temp_pos-4]
print(Detuning)

In [ ]:
# Voltages associated with each condition
# In names and files each voltage is associated with numbers
# For instance on epoint is labeled DC03_RF18
# You need to know which voltage is DC03 and RF18

# You have to look on the Fortran code, example :
#    !dec$ elseif defined(Vrf18)
#    double precision  , parameter :: V_rf  = 7.179d1
#    character(len=100), parameter :: str_extra3 = 'q=0.666'
# Here RF18 is 71.79 V, q = 0.666

Udc = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 7.5, 3.33, 3.66, 4.5, 5.33, 5.66, 6, 6.5]
Urf = [10.77, 16.15, 21.54, 26.92, 32.31, 37.69, 43.08, 48.46, 53.85, 59.23, 64.61,
       70.00, 75.38, 80.77, 61.02, 62.82, 66.41, 68.20, 71.79, 55.64, 57.43]
len(Udc),len(Urf)

In [ ]:
# calcul des paramètres Mathieu et frequences propres

wx = np.zeros((len(points_and_coord)))
wz = np.zeros((len(points_and_coord)))
w_ratio = np.zeros((len(points_and_coord)))

for i,j in enumerate(points_and_coord):
    # voltage recovery
    coucou_DC = int(j[j.find('DC')+2:j.find('DC')+4])
    coucou_RF = int(j[j.find('RF')+2:j.find('RF')+4])
    # print(Udc[coucou_DC],'   ',Urf[coucou_RF])
    ax = 4*(C_e/m_Ca)*Udc[coucou_DC]/(r0**2*Omega**2)
    az = 8*mkappa*(C_e/m_Ca)*Udc[coucou_DC]/(d0**2*Omega**2)
    qx = -4*(C_e/m_Ca)*Urf[coucou_RF]/(r0**2*Omega**2)
    wx[i] = Omega/2*sqrt(qx**2/2+ax-0.5*az)/(2*pi)
    wz[i] = sqrt(2*mkappa*C_e*Udc[coucou_DC]/(m_Ca*d0**2))/(2*pi)
    w_ratio[i] = wz[i]/wx[i]
    print(i,w_ratio[i])

# Plots
Nuage d'ions (XYZ) et température

# One try study

In [78]:
# Choose point to plot

# point number
plot_point = 0
plot_try = 0
plot_subdir = all_subdir[plot_point*13+plot_try%13] # point address

coucou_cond = list(points_and_coord.keys())[plot_point] # point condition
print('>',coucou_cond) 
# voltage recovery
the_DC = int(coucou_cond[coucou_cond.find('DC')+2:coucou_cond.find('DC')+4])
the_RF = int(coucou_cond[coucou_cond.find('RF')+2:coucou_cond.find('RF')+4])

print('Udc  ','Urf')
print(Udc[the_DC],'   ',Urf[the_RF])
#choose try

print('>',plot_subdir)

> DC03_RF19
Udc   Urf


NameError: name 'Udc' is not defined

In [ ]:
# plot ion cloud XYZ

print(plot_subdir)
# pick the right xyz files
onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]
file_name = '{}/xva{}'.format(plot_subdir,sort(onlyfiles)[0].strip('.dat')[4:])
plot_XYZ(file_name,fig_name='XYZ_point'+str(plot_point)+'_try'+str(plot_try),fig_title='XYZ')
auxr_LC,auxv_LC,auxa_LC = load_xyz_init_bin_DP(file_name)
len(auxr_LC[0])

In [ ]:
for k,l in enumerate(all_subdir):
    print(k,l[55:])

In [ ]:
# point number
plot_point = 0
plot_try = 0
plot_subdir = all_subdir[0] # point address

In [79]:
plot_subdir

'/home/adrian/Documents/Programmes/Fortran_Jofre/20200609/DC03_RF19/Try00'

In [70]:
# Plot T and fluo PM for RF relax POST INJECT

onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]
ta, te, ti, to, tu = plot_T_and_PM_Init_RFrelax_AfterInj_Evol( plot_subdir+'/',sort(onlyfiles)[0].strip('.dat')[4:],1,'coucou',xlim1=(3.42,4.45),ylim1=(5e-5,3e3))
gcf().set_size_inches(11.69, 8.27)
gcf().subplots_adjust(hspace=0.015)
# ,xlim1=(3.458,3.582),ylim1=(2e-4,50),ylim2=(19.5,65)

/home/adrian/.local/lib/python3.6/site-packages/matplotlib/axes/_base.py:3449: UserWarning: Attempted to set non-positive ylimits for log-scale axis; invalid limits will be ignored.
  'Attempted to set non-positive ylimits for log-scale axis; '


In [65]:
# Plot T and fluo PM for RF relax POST COOLING

onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]
ta, te, ti, to, tu = plot_T_and_PM_Init_RFrelax_AfterCooling_Evol( plot_subdir+'/',sort(onlyfiles)[0].strip('.dat')[4:],1,'coucou',xlim1=(-1.5,55),ylim1=(5e-5,12e3))
gcf().set_size_inches(11.69, 8.27)
gcf().subplots_adjust(hspace=0.015)
# ,xlim1=(3.458,3.582),ylim1=(2e-4,50),ylim2=(19.5,65)

/home/adrian/.local/lib/python3.6/site-packages/matplotlib/axes/_base.py:3449: UserWarning: Attempted to set non-positive ylimits for log-scale axis; invalid limits will be ignored.
  'Attempted to set non-positive ylimits for log-scale axis; '


NameError: name 'PM_variation' is not defined

In [ ]:
dim_nu[4,0,0]

In [ ]:
rtoLratio = []
for k,l in enumerate(all_subdir):
        rtoLratio.append( dim_nu[k,0,0] / dim_nu[k,0,2])
print(rtoLratio)

In [ ]:
# determine how much ions for one point (one condition)
print('Udc  ','Urf')
print(Udc[coucou_DC],'   ',Urf[coucou_RF])
for plot_try in [3,4,8,9]:
    plot_subdir = all_subdir[plot_point*13+plot_try%13]
    onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]
    file_name = '{}/xva{}'.format(plot_subdir,sort(onlyfiles)[0].strip('.dat')[4:])
    plot_XYZ(file_name,fig_name='XYZ_point'+str(plot_point)+'_try'+str(plot_try),fig_title='XYZ')
    auxr_LC,auxv_LC,auxa_LC = load_xyz_init_bin_DP(file_name)
    print(len(auxr_LC[2]))

In [ ]:
# Plot T and fluo PM

plot_subdir = all_subdir[plot_point*13+plot_try%13] # point address
onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]

file_dir2 = plot_subdir+os.sep
file_name = sort(onlyfiles)[k].strip('.dat')
flag_plot = 1
fig_name = 'coucou'
    
            
i_aux = file_name.find('_N')
file1 = 'SimuType0'    + file_name[i_aux:] ########## file1 = 'SimuType0'    + file_name[i_aux:17+36]
file2 = 'SimuType4_01' + file_name[i_aux:]
file3 = 'SimuType2_01' + file_name[i_aux:]

tt1, T_CM1, T_aux1, PM1 = load_T_and_PM_simu(file_dir2+'Temp_'+file1)
tt2, T_CM2, T_aux2, PM2 = load_T_and_PM_simu(file_dir2+'Temp_'+file2+'50eV')
tt3, T_CM3, T_aux3, PM3 = load_T_and_PM_simu(file_dir2+'Temp_'+file3+'50eV')

aux = mean(PM1[-100:])
PM_variation = ( aux - mean(PM3[-100:]) ) / aux
T_variation  = mean(T_aux3[-100:,0]) + mean(T_aux3[-100:,1]) + mean(T_aux3[-100:,2])


# Auxiliary arrays:
t_aux1 = array([tt2[ 0],tt2[ 0]])
t_aux2 = array([tt2[-1],tt2[-1]])
y1_aux = array([1.0e-3 ,1.0e-1 ])
#     y2_aux = array([0 ,20 ])
y2_aux = array([0 ,50 ])

tt    = concatenate( (   tt1,   tt2,   tt3) )
T_aux = concatenate( (T_aux1,T_aux2,T_aux3) )
PM    = concatenate( (PM1,PM2,PM3) )

if flag_plot == 1 :
    #fig_name = file_name[-9:]
    figure(fig_name); clf()
    ax1 = subplot(211)
    semilogy(tt*1.e3,T_aux[:,0], label='Tx')
    semilogy(tt*1.e3,T_aux[:,1], label='Ty')
    semilogy(tt*1.e3,T_aux[:,2], label='Tz')
    semilogy(t_aux1*1.e3,y1_aux,'r')
    semilogy(t_aux2*1.e3,y1_aux,'r')
    ax1.grid()

    legend()
    # ~ xlabel('time[ms]')
    # ~ ylabel('T[K]')
    plt.setp(ax1.get_xticklabels(),visible=False)

    ax2 = subplot(212,sharex=ax1)
    plot(tt*1.e3,PM[:])
    plot(t_aux1*1.e3,y2_aux,'r')
    plot(t_aux2*1.e3,y2_aux,'r')
    ax2.grid()

    xlabel('time[ms]')
    ylabel('Counts')
    
    ax1.annotate(list(points_and_coord.keys())[plot_point], (3.45, 10),color='xkcd:black',fontsize=13,
        ha='left',va='center',weight='bold',zorder = 1000)
    ax1.annotate('pt'+str(plot_point), (3.45, 3.5),color='xkcd:black',fontsize=13,
        ha='left',va='center',weight='bold',zorder = 1000)

    ax1.set_xlim(3.445,3.580)
    ax1.set_ylim(0.1e-3,100)
    ax2.set_ylim(-2,90)
    plt.tight_layout()
    subplots_adjust(hspace=0.015)
    
    savefig(str(file_path[3:11])+str(list(points_and_coord.keys())[plot_point]),dpi=100)

In [ ]:
# Plot only T with simulation time step
fname = 'only T'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
ax.plot(ta*1000,ti)
ax.grid()

ax.set_xlabel('[ms]')
ax.set_ylabel('[cts/500ns]')
tight_layout()
 
# savefig('ex_simu_clic_tr6_dc4_rf5_zoom.png',dpi=300)

In [ ]:
# Plot only T with custom time step (realistic PM behaviour)

time_step = 1e-3 # s
custom_pts = len(ta)*(ta[1]-ta[0])/time_step
print(custom_pts)
somme_pts = int( round( time_step/(ta[1]-ta[0]) ) )
print(somme_pts)

custom_t = np.linspace(1,ceil(max(ta)/time_step),custom_pts+1)
custom_t_offset = max(custom_t) - max(ta)/time_step
custom_t = custom_t - custom_t_offset
custom_fluo = np.zeros( int(ceil(custom_pts)) )

for j in range( int(ceil(custom_pts))-1 ):
    temp = []
    print(j)
    print(somme_pts*j,somme_pts*(j+1)-1)
    for k in range(somme_pts*j,somme_pts*(j+1)):
        temp.append(ti[-k-1])
    temp = sum(temp)
    custom_fluo[-1-j] = temp/max(ti)/15

fname = 'only T custom T step'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
ax.plot(ta*1000,ti,zorder=10,label='500 ns')
# for j in range( int(ceil(custom_pts)) ):
#     ax.plot(ta[::-1][somme_pts*j:somme_pts*(j+1)]*1000,ti[::-1][somme_pts*j:somme_pts*(j+1)])
ax.scatter(custom_t[1:],custom_fluo[1:]/max(custom_fluo)*max(ti),s=200,marker='+',lw=5,edgecolor='black',zorder=100,label='1 ms')

ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
ax.grid()


ax.set_xlim(1.1,5.98)
ax.set_xlabel('[ms]')
ax.set_ylabel('[cts]')
ax.legend(title='Time step')
tight_layout()
 
# savefig('ex_simu_clic_tr6_dc4_rf5_zoom.png',dpi=300)

## Data visualisation for all points (SNR)

In [ ]:
# A 3D visualisation of SNR for each point, each run
# Compulsory because this is where SNR is put in shape (under the name Z)

ax = 0
ay = 0
az, qx, wx, wy, wz = [],[],[],[],[]

fname = '3D_GMol_var_fluo_contour'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111, projection='3d')

# fig1 = plt.figure(num=fname,clear=True)
# ##### PLOT DFluo et Proba détection
# ax1 = fig1.add_subplot(111)
# ax2 = ax1.twinx()
# clf()

l = 0
plot_Udc = []
plot_Urf = []
Z = []
for i,j in enumerate(points_and_coord):
    plot_Udc.append(Udc[int(points_and_coord[j][0])])
    plot_Urf.append(Urf[int(points_and_coord[j][1])])
    Z.append(Gmol_data[2][i])
            # ax.set_aspect('equal')
Z = np.array(Z)
print(shape(plot_Udc),shape(plot_Urf),shape(Z))

for k in range(13):
    ax.scatter(xs=plot_Udc,ys=plot_Urf,zs=Z[:,k],color='xkcd:azul',marker='o',s=10)
#             ax.plot(xs=r_LC_clip[0],ys=r_LC_clip[2],zs=r_LC_clip[1],marker='o',ms=0.45,ls='',color='xkcd:blue with a hint of purple')

#             ax.view_init(elev=18, azim=55)
#             ax.set_xlim(-3e-1,3e-1)
#             ax.set_ylim(-3e-1,3e-1)
#             ax.set_zlim(-3e-1,3e-1)

#             ax.text2D(0.05, 0.85, f'$U_{{ RF }}$ =  {Urf} V\n$U_{{DC}}$ = {Udc[j]} V', transform=ax.transAxes)
ax.set_xlabel('\n$U_{DC}$ [V]')
ax.set_ylabel('\n$U_{RF}$ [V]')
ax.set_zlabel('\n SNR ')

#         ax.plot(np.ones(100)*Udc[dc], np.ones(100)*Urf[rf], zs=linspace(0,0.8,100),color='black',ls=':',lw=1)

# X = [Udc for hhh in range(elem_1+1)]
# Y = [Urf for smeuiofh in range(elem_2-1)]
# # X, Y = np.meshgrid(X, Y)
# Z = asarray(transpose(mean_fluo))
# Z = np.meshgrid(Z)

# # surf = ax.plot_wireframe(transpose(X), Y, Z)
# surf = ax.plot_surface(transpose(X), Y, Z, rstride=1, cstride=1,
#                 cmap='viridis', edgecolor='black',alpha = 0.35)
# #         ax.errorbar(Udc[dc],mean_fluo[],yerr=prob_detect_xerr,xerr=None,
# #              elinewidth=2.5,capsize=None,color='xkcd:yellow orange',
# #              fmt=':+',ms=12,mew=3.5,mfc='None',mec='xkcd:yellow orange')
# cbar = plt.colorbar(surf)
# cbar.ax.set_ylabel(r'$\Delta F / F$ moyen ('+str(elem_0)+' essais)', rotation=90)


fig.set_size_inches(11.69, 8.27)
fig.tight_layout()
# subplots_adjust(hspace=0.015)

# savefig(fname,dpi=300)

## 2D plot

In [ ]:
x = []
y = []
z = []
for g,h in enumerate(points_and_coord):
    x.append(Udc[int(points_and_coord[h][0])])
    y.append(Urf[int(points_and_coord[h][1])])
    z.append(mean(trig_detection[g]))
    
fname = '2D_GMol_var_fluo_contour'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
title(all_subdir[0][:-15],fontsize=25)

im = ax.scatter(x,y,
        marker='o',s=400,edgecolor='black',zorder = 1000,
        c = z, cmap='rainbow')
ax.set_xlabel(r'$U_{DC}$ [V]')
ax.set_ylabel(r'$U_{RF}$ [V]')
ax.grid()

for i, txt in enumerate(points_and_coord):
    ax.annotate(str(i), (x[i], y[i]),color='xkcd:black',fontsize=13,
                ha='center',va='center',weight='bold',zorder = 1000)
    # ax.annotate(str(txt), (x[i], y[i]-0.70),fontsize=9)
    if mean(cool_cloud[i])*13 < 13:
        ax.annotate(f'{mean(cool_cloud[i])*13:.0f}',(x[i]-0.12, y[i]+0.25),fontsize=13,color='red',ha='right')
    #ax.annotate(str(txt)+'\n'+str(i), (x[i], y[i]))

cbar = plt.colorbar(im, ax=ax)
cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(SNR_trig_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
# ax.grid()
ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'.png',dpi=300)

# More specific visualisations

## One condition (one RF or DC ...)

In [ ]:
print('===== Udc =====')
for a,b in enumerate(Udc):
    print(' <>',f'{a:02d}',':',f'{b:.2f}',end='')
print('\n===== Urf =====')
for a,b in enumerate(Urf):
    print(' <>',f'{a:02d}',':',f'{b:.2f}',end='')

In [ ]:
# Choose a specific condition to consider
# Choose one line on the plot (one RF)
condition_to_respect = 'RF10'
points_to_plot = [k for k,l in enumerate(list(points_and_coord.keys())) if condition_to_respect in l]
print(points_to_plot)

In [ ]:
# plot delta E Gmol absolu for one RF voltage

SNR_trig_threshold = 1
shapevar = ((len(points_and_coord), len(num_runs)))

ax = 0
ay = 0
az, qx, wx, wy, wz = [],[],[],[],[]
alpha_dre = zeros(shapevar)
rtoLratio = zeros(shapevar)

fname = 'Fluo_stat_all'
fig1 = plt.figure(num=fname,clear=True)
gs = gridspec.GridSpec(2, 2, width_ratios=[3, 1]) 
##### PLOT DFluo et Proba détection
ax1 = fig1.add_subplot(gs[0])
ax2 = ax1.twinx()
# clf()

DC_index = list(points_and_coord)[0].find('DC')
RF_index = list(points_and_coord)[0].find('RF')
plot_Udc = [Udc[ int(list(points_and_coord)[x][DC_index+2:DC_index+4])] for x in points_to_plot]
plot_Urf = [Urf[ int(list(points_and_coord)[x][RF_index+2:RF_index+4])] for x in points_to_plot]

suptitle(all_subdir[0][:-15]+'  | Urf='+str(plot_Urf[0])+', D=-'+str(Detuning),fontsize=12)

# az.append(8*mkappa*(C_e/m_Ca)*plot_Udc/(d0**2*Omega**2))
# qx.append(-4*(C_e/m_Ca)*plot_Urf/(r0**2*Omega**2))

#     # frequences attendues
#     wx.append(Omega/2*sqrt(qx[l]**2/2+ax-0.5*az[l])/(2*pi))
# #     wy.append(Omega/2*sqrt(qx[l]**2/2+ay-0.5*az[l])/(2*pi))
#     wz.append(sqrt(2*mkappa*C_e*plot_Udc[i]/(m_Ca*d0**2))/(2*pi))
    
for i,j in enumerate(points_to_plot):
    # a,q attendu       
    SNR = Gmol_data[2][j]
    ax1.plot([plot_Udc[i]]*len(SNR),SNR,'o',color='xkcd:azul',markerfacecolor='none')
    
    prob_detect_xerr = sqrt(mean(trig_detection,axis=1)[j]*(1-mean(trig_detection,axis=1)[j]))/sqrt(len(SNR))
    ax2.errorbar(plot_Udc[i],mean(mean(trig_detection,axis=1)[j]),yerr=prob_detect_xerr,xerr=None,
             elinewidth=3,capsize=None,color='xkcd:black',
             fmt=':+',ms='8',mew=3,mfc='None',mec='xkcd:red orange')
#     annotate('qx={:1.1e}\nUrf={:3.1f}V\nwx={:3.1f}kHz'.format(qx[0],Urf_no_repeat,wx[0]/1000), xy=(2,0.25), xycoords='data',
#         size=10, ha='center', va='top', color='xkcd:azul',
#         bbox=dict(boxstyle='round', fc='none',edgecolor='xkcd:azul'))
    
    l+=1
    
# for k in range(0,len(Udc_no_repeat),3):
#         ax1.annotate('a={:1.1e}\nwz={:3.1f}kHz'.format(az[k],wz[k]/1000), xy=(Udc[k],0.73), xycoords='data',
#         size=8, ha='center', va='top', color='xkcd:azul',
#         bbox=dict(boxstyle='round', fc='none',edgecolor='none')) 
       
ax1.set_ylabel('$\Delta F$/Bruit', color='xkcd:azul')
ax2.set_ylabel('Taux de détection', color='xkcd:red orange') # P_{Detect} +' ('+str(len(data0[1]))+' runs)'
ax1.tick_params(axis='y', labelcolor='xkcd:azul')
ax2.tick_params(axis='y', labelcolor='xkcd:red orange')

ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
ax1.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
ax2.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.2))
ax1.set_ylim(-0.2,5.35)
ax2.set_ylim(-0.2/5,5.35/5)
ax1.grid()

##### Plot DEnergie et moyenne de DEnergie
renorm = 0.5*m_GM/C_e
ax3 = fig1.add_subplot(gs[2],sharex=ax1)
ax4 = ax3.twinx()
# clf()
for i,j in enumerate(points_to_plot):
    ax3.plot([plot_Udc[i]]*len(deltaEc[j][:]),deltaEc[j][:]*renorm*1000,'o',color='xkcd:azul',markerfacecolor='none')

mean_deltaEc = [mean(deltaEc[j][:])*renorm*1000 for i,j in enumerate(points_to_plot)]
        
ax4.plot(plot_Udc,mean_deltaEc,linestyle='',marker='+',color='xkcd:red orange',ms=8,mew=3)
ax4.annotate('$E_{GMol}$=50 eV', xy=(1,5), xycoords='data',
    size=12, ha='center', va='top', color='xkcd:azul',
    bbox=dict(boxstyle='round', fc='none',edgecolor='xkcd:azul')) 

ax3.set_xlabel('$U_{DC}$')
ax3.set_ylabel('$\Delta E$ [meV]', color='xkcd:azul')
ax4.set_ylabel('$\Delta E$ moyen [meV]', color='xkcd:red orange')
ax3.tick_params(axis='y', labelcolor='xkcd:azul')
ax4.tick_params(axis='y', labelcolor='xkcd:red orange')

ax3.set_ylim(-70,7)
ax4.set_ylim(-70,7)
ax3.grid()

ax3.set_xlim(-0.1,8.1)

ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
plt.setp(ax1.get_xticklabels(), visible=False)

##### Plot R/L et alpha
ax5 = fig1.add_subplot(gs[1],sharex=ax1)
ax6 = fig1.add_subplot(gs[3],sharex=ax5)

rtoLratio = [[] for k in range(len(points_to_plot))]
for i,j in enumerate(points_to_plot):
    for k,l in enumerate(dim_nu[j,:,0]):
        rtoLratio[i].append( dim_nu[j,k,0] / dim_nu[j,k,2])
mean_rtoLratio = [mean(rtoLratio[i][:]) for i,j in enumerate(points_to_plot)]
for i,j in enumerate(points_to_plot):
    ax5.plot([plot_Udc[i]]*len(rtoLratio[i][:]),rtoLratio[i][:],
             'o',color='xkcd:azul',markerfacecolor='none')
    ax6.plot(plot_Udc[i],w_ratio[j],
             'o',color='xkcd:azul',markerfacecolor='none',mew = 2)
    
ax5.plot(plot_Udc,mean_rtoLratio,
         linestyle='',marker='o',mew = 2,color='xkcd:red orange',markerfacecolor='none')

ax6.set_xlabel('$U_{DC}$')
ax5.set_ylabel('R/L')
ax6.set_ylabel(r'$\alpha = \omega_z / \omega_r$')
# ax5.tick_params(axis='y', labelcolor='xkcd:azul')
# ax6.tick_params(axis='y', labelcolor='xkcd:yellow orange')

ax6.set_xlim(-0.1,8.1)
ax5.set_ylim(0.07,0.51)
ax6.set_ylim(0.07,0.51)
ax5.grid()
ax6.grid()

ax5.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=2))
ax5.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))
ax6.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))
plt.setp(ax5.get_xticklabels(), visible=False)

tight_layout()

fig1.set_size_inches(11.69, 8.27)
fig1.tight_layout()
subplots_adjust(hspace=0.015,top=0.935)

savefig(fname,dpi=300)

## One point

In [ ]:
# Choose a specific point to consider
# Choose one point on the plot
condition_to_respect = 'DC03_RF10'
address_to_plot = [l for k,l in enumerate(all_subdir) if condition_to_respect in l]
points_to_plot  = [k for k,l in enumerate(list(points_and_coord.keys())) if condition_to_respect in l] # point number
print(address_to_plot,points_to_plot)
whatisthisfile = re.sub('/','-',file_path[3:21])
whatisthisfile = condition_to_respect

In [ ]:
# y_hist_detec[i]
renorm = 0.5*m_GM/C_e
deltaEc[points_to_plot][0]*renorm*1000

In [ ]:
# histogram for Delta energy
# two families : detection and non detection
point_SNR = Gmol_data[2][points_to_plot][0]
y_hist_detec    = [l*renorm*1000 for k,l in enumerate(deltaEc[points_to_plot][0]) if abs(point_SNR[k]) > SNR_trig_threshold]
y_hist_nondetec = [l*renorm*1000 for k,l in enumerate(deltaEc[points_to_plot][0]) if abs(point_SNR[k]) < SNR_trig_threshold]

fname = 'Histogram_Delta_energy_GMol'
fig = plt.figure(num=fname, clear=True)
# plt.suptitle('Histogram $\Delta E$')
# gs = gridspec.GridSpec(2, 2, width_ratios=[3, 1]) 
##### PLOT DFluo et Proba détection
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212,sharex=ax1)
 # PMvar[k][my_selection]
my_bins = arange(-60,0,2)
ax1.set_title(r'Detection SNR>'+str(SNR_trig_threshold),color='xkcd:leaf green')
ax1.hist(y_hist_detec,bins=my_bins ,color='xkcd:leaf green', edgecolor='black', linewidth=1.2)
ax2.set_xlabel(r'$\Delta E$ [meV]')
ax1.set_ylim(0,3)
ax1.grid()
str_stat
ax1.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
ax1.annotate(f'{len(y_hist_detec)} cases', xy=(-60,2.5), xycoords='data',
    size=12, ha='left', va='top', color='xkcd:leaf green',
    bbox=dict(boxstyle='round', fc='white')) 

ax2.set_title(r'No detection SNR>'+str(SNR_trig_threshold),color='xkcd:rouge')
ax2.hist(y_hist_nondetec,bins=my_bins ,color='xkcd:rouge', edgecolor='black', linewidth=1.2)
ax2.set_ylim(0,3)
ax2.grid()
ax2.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
ax2.annotate(f'{len(y_hist_nondetec)} cases', xy=(-60,2.5), xycoords='data',
    size=12, ha='left', va='top', color='xkcd:rouge',
    bbox=dict(boxstyle='round', fc='white')) 

fig.set_size_inches(11.69, 8.27)
fig.tight_layout()
subplots_adjust(hspace=0.35,top=0.94)

# savefig(fname+'_D'+str(Detuning)+'_'+str(whatisthisfile)+'.png', dpi=300)

In [ ]:
# plot DE vs Udc but with red green colors

# my_order = [0,1,2,3,4,5,6,13,7,8,9,14,15,10,16,11,12] # 13,7,8,9,14,15,10,16,11,12

# Udc = [0.5,1,1.5,2,2.5,3,3.5,3.75,4,4.5,5,5.3,5.7,6,6.5,7,8]
# Urf = 64.61

mean_fluo = [mean(PMvar[:,j]) for j in my_order]

fig1 = plt.figure(num='Fluo var only_DE_colors',clear=True)

##### PLOT DFluo et Proba détection
ax1 = fig1.add_subplot(111)
ax2 = ax1.twinx()

left, bottom, width, height = [0.4145, 0.12, 0.45, 0.5] # cadre zoom
axins = fig1.add_axes([left, bottom, width, height])

my_color = ['xkcd:rouge', 'xkcd:leaf green']
l = 0
for j in range(len(Udc)):          
    # a,q attendu
    az.append(8*mkappa*(e/m)*Udc[j]/(d0**2*Omega**2))
    qx.append(-4*(e/m)*Urf/(r0**2*Omega**2))

    # frequences attendues
    wx.append(Omega/2*sqrt(qx[l]**2/2+ax-0.5*az[l])/(2*pi))
#     wy.append(Omega/2*sqrt(qx[l]**2/2+ay-0.5*az[l])/(2*pi))
    wz.append(sqrt(2*mkappa*e*Udc[j]/(m*d0**2))/(2*pi))
    
    for k in range(elem_0):
        trig = 0
        if PMvar[k][my_order[j]] > 0.6:
            sigdetect[j] +=1
            trig = 1
        alpha_dre[k][j] = wz[j]**2/wx[j]**2
        rtoLratio[k][j] = dim_nu[k,j,0] / dim_nu[k,j,2]
        ax1.plot(Udc[j],deltaEc[k][my_order[j]]*renorm*1000,'o',color=my_color[trig],markerfacecolor='none')
        axins.plot(Udc[j],deltaEc[k][my_order[j]]*renorm*1000,'o',color=my_color[trig],markerfacecolor='none')
    
    l+=1


mean_deltaEc = [mean(deltaEc[:,j])*renorm*1000 for j in my_order]
        
ax2.plot(Udc,mean_deltaEc,linestyle=':',marker='>',color='xkcd:yellow orange',markerfacecolor='none')
ax2.annotate('$E_{GMol}$=50 eV', xy=(1,-5), xycoords='data',
    size=10, ha='center', va='top', color='xkcd:azul',
    bbox=dict(boxstyle='round', fc='none',edgecolor='xkcd:azul')) 

ax1.set_xlabel('$U_{DC}$')
ax1.set_ylabel('$\Delta E$ [meV]', color='xkcd:azul')
ax2.set_ylabel('mean $\Delta E$ [meV]', color='xkcd:yellow orange')
ax1.tick_params(axis='y', labelcolor='xkcd:azul')
ax2.tick_params(axis='y', labelcolor='xkcd:yellow orange')

ax1.set_ylim(-0.10*1000,1)
ax2.set_ylim(-0.10*1000,1)
ax1.grid()

ax2.set_xlim(-0.1,8.5)

ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
ax1.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=10))

x1, x2, y1, y2 = 3.25, 8.25, -21, -2.5 # limites zoom
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.grid()

mark_inset(ax1, axins, loc1=2, loc2=1, fc="none", ec="0.4")
# axins.invert_yaxis()
plt.setp(axins.get_xticklabels(), visible=False)
plt.setp(axins.get_yticklabels(), fontsize=10)


fig1.set_size_inches(11.69, 8.27)
fig1.tight_layout()

# Trajectory
Trajectoire GMol

In [ ]:
# Recover all GMol trajectories addresses
# + compare bi for-loop with nested comprehension list

# Soit A
trajectory_file  = []
temperature_file = []
for j,k in enumerate(address_to_plot):
    temp_files = listdir(k)
    for l,m in enumerate(temp_files):
        if 'trj' in m:
            trajectory_file.append(m)
        if 'Temp_SimuType4' in m:
            temperature_file.append(m)

# Soit B
trajectory_files  = [[m for l,m in enumerate(temp_files) if 'trj' in m] for j,k in enumerate(address_to_plot)]
temperature_files = [[m for l,m in enumerate(temp_files) if 'Temp_SimuType4' in m] for j,k in enumerate(address_to_plot)]
    
#trajectory_file = [listdir(j) for i,j in enumerate(address_to_plot)]
#trajectory_file = [k for j,k in enumerate(trajectory_file) if 'trj' in k]
print(trajectory_file)

In [ ]:
temp_size = len(load_trj(os.path.join(address_to_plot[0],trajectory_files[0][0]).strip('.bin'))[0])
t, r_x,r_y,r_z, v_x,v_y,v_z, a_x,a_y,a_z = np.zeros((10,len(trajectory_files),temp_size))
tt, PM      = np.zeros((2,len(temperature_files),temp_size))
T_CM, T_aux = np.zeros((2,len(temperature_files),temp_size,3))
for j,k in enumerate(trajectory_files): # try
    print('>',j,'-',k)
    t[j], r_x[j],r_y[j],r_z[j], v_x[j],v_y[j],v_z[j], a_x[j],a_y[j],a_z[j] = load_trj(os.path.join(address_to_plot[j],k[0].strip('.bin')))
for j,k in enumerate(temperature_files): # try
    print('>',j,'-',k)
    tt[j], T_CM[j], T_aux[j], PM[j] = load_T_and_PM_simu(os.path.join(address_to_plot[j],k[0].strip('.dat')))

In [ ]:
Ec_GMol = np.zeros((len(trajectory_files),temp_size))
for walay,bilai in enumerate(trajectory_files):
    for couscous,merguez in enumerate(t[walay]):
        Ec_GMol[walay,couscous] = 0.5*m_GM*(v_x[walay,couscous]**2+v_y[walay,couscous]**2+v_z[walay,couscous]**2)

In [ ]:
# plot energy evolution of GMol
# during fly time
# depending time and space
cm = pylab.get_cmap('rainbow')   # colormap par défaut color=cm(k/len(array))

fname = 'GMol_energy_time'
fig = plt.figure(fname,clear=True)
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
#title(all_subdir[0][:-15],fontsize=12)

for i in range(13):
    #i = 0
    w = t[i]
    x = r_z[i]
    y = Ec_GMol[i]
    cloud_size = dim_nu[points_to_plot,i,2]

    im = ax1.scatter(w[22:40]*1000,y[22:40]/C_e,
            marker='+',s=42,color=cm(i/12))
    ax1.set_xlabel(r't [ms]')
    ax1.set_ylabel(r'Energie [eV]')
    ax1.grid()

    im = ax2.scatter(x[22:40]*1000,y[22:40]/C_e,
            marker='+',s=42,color=cm(i/12))
    ax2.set_xlabel(r'r_z [mm]')
    ax2.set_ylabel(r'Energie [eV]')
    ax2.grid()

    ax2.vlines(-cloud_size,49.92,49.995,color='gray',linestyle=':',lw=2)
    ax2.vlines(cloud_size,49.92,49.995,color='gray',linestyle=':',lw=2)
# ax.grid()
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
#savefig(fname+'_D'+str(Detuning)+'.png',dpi=300)

In [ ]:
# plot energy evolution of GMol
# during fly time
# depending space
cm = pylab.get_cmap('rainbow')   # colormap par défaut color=cm(k/len(array))

fname = 'GMol_energy_pos'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
#title(all_subdir[0][:-15],fontsize=12)

for i in range(13):
    w = t[i]
    x = r_z[i]
    y = Ec_GMol[i]
    cloud_size = dim_nu[points_to_plot,i,2]
    im = ax.scatter(x[22:40]*1000,y[22:40]/C_e,
            marker='+',s=55,lw=3,color=cm(i/13))
    ax.set_xlabel(r'r_z [mm]')
    ax.set_ylabel(r'Energie [??]')
    ax.grid()
    

ax.vlines(-cloud_size,49.92,49.995,color='gray',linestyle=':',lw=2)
ax.vlines(cloud_size,49.92,49.995,color='gray',linestyle=':',lw=2)       
    # ax.grid()
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
    # savefig(fname+'_D'+str(Detuning)+'.png',dpi=300)

In [ ]:
rz_0 = -1.5e-3
eta = 1. / (4.*(sqrt(10) + 5.))
wz_LC = 2*pi*90806.9982303
wz2 = wz_LC**2 * m_Ca / m_GM
L     = 0.014
rz  = np.linspace(rz_0*0.35,-rz_0*0.35,1000)
# Ep = wz2*eta*L**2*m_GM*(1- exp(rz_0**2 / (2*eta*L**2)) )
my_cte = -1/(2*eta*L**2)
Ep = wz2*eta*L**2*m_GM*(1-exp(my_cte*rz**2))/C_e*2
Ep0   = wz2*eta*L**2*m_GM*(1- exp(rz_0**2 / (2*eta*L**2)) )

In [ ]:
# plot energy evolution of GMol
# during fly time
# depending space
cm = pylab.get_cmap('gist_rainbow')   # colormap par défaut color=cm(k/len(array))
my_marker = ['+','x','3','4']
trig_marker     = 'o'
nontrig_marker = 'x'

fname = 'GMol_energy_pos_allsublots'
fig, ax = plt.subplots(2, 2,num=fname,clear=True,sharex=True,sharey=True)
#title(all_subdir[0][:-15],fontsize=12)

for i in range(13-1):
    j = i//3 %2 #i%7
    k = i//6 %2
    w = t[i]
    x = r_z[i]
    y = Ec_GMol[i]
    yy = -Ep*1.86 + 50
    cloud_size = dim_nu[points_to_plot,i,2]
    
    ax[j,k].vlines(-cloud_size*1000,49.91,50.05,color='xkcd:charcoal',linestyle=':',lw=2,zorder=0)
    ax[j,k].vlines(cloud_size*1000,49.91,50.05,color='xkcd:charcoal',linestyle=':',lw=2,zorder=0)  
    
    if abs(point_SNR[i]) > SNR_trig_threshold:
        the_marker = trig_marker
        im = ax[j,k].scatter(x[22:40]*1000000,y[22:40]/C_e,
            marker=the_marker,s=55,lw=1.8,color='',edgecolor=cm(i/13))
        im = ax[j,k].scatter(x[22:40]*1000000,y[22:40]/C_e,
            marker='+',s=55,lw=1,color=cm(i/13))
    else:
        the_marker = nontrig_marker
        im = ax[j,k].scatter(x[22:40]*1000000,y[22:40]/C_e,
            marker=the_marker,s=55,lw=3,color=cm(i/13))
    
    ax[j,k].plot(rz*1000000,yy,
            lw=2,color='gray',zorder = 0)
    ax[j,k].grid()
    #plt.setp(ax[j,k].get_xticklabels(), visible=False)
    ax[j,k].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=200))
    ax[j,k].yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.02))
    ax[j,k].set_ylim(49.915,50.005)
    # ax.grid()
    
i = 12    
j = 1 #i%7
k = 1
w = t[i]
x = r_z[i]
y = Ec_GMol[i]
cloud_size = dim_nu[points_to_plot,i,2]
if abs(point_SNR[i]) > SNR_trig_threshold:
    the_marker = trig_marker
    im = ax[j,k].scatter(x[22:40]*1000000,y[22:40]/C_e,
        marker=the_marker,s=55,lw=1.8,color='',edgecolor=cm(i/13))
    im = ax[j,k].scatter(x[22:40]*1000000,y[22:40]/C_e,
        marker='+',s=55,lw=1,color=cm(i/13))
else:
    the_marker = nontrig_marker
    im = ax[j,k].scatter(x[22:40]*1000000,y[22:40]/C_e,
        marker=the_marker,s=55,lw=3,color=cm(i/13))
ax[j,k].vlines(-cloud_size*1000,49.91,50.05,color='xkcd:charcoal',linestyle=':',lw=2,zorder=0)
ax[j,k].vlines(cloud_size*1000,49.91,50.05,color='xkcd:charcoal',linestyle=':',lw=2,zorder=0)  
#plt.setp(ax[j,k].get_xticklabels(), visible=False)
ax[j,k].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=200))
ax[j,k].yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.02))
ax[j,k].set_ylim(49.915,50.005)
   
#ax[-1, -1].set_visible(False)
ax[1,0].set_xlabel(r'r_z [µm]')
ax[1,1].set_xlabel(r'r_z [µm]')
ax[1,0].set_ylabel(r'Energie [eV]')
ax[0,0].set_ylabel(r'Energie [eV]')
fig.suptitle('Energie cinétique de la GMol durant sa traversée')
#plt.setp(ax[6,0].get_xticklabels(), visible=True)
#plt.setp(ax[5,1].get_xticklabels(), visible=True,zorder=10000)

fig.set_size_inches(13.5, 10)

fig.tight_layout()
subplots_adjust(hspace=0.013,wspace=0.015,top=0.93)
savefig(fname+'_D'+str(Detuning)+'_'+str(whatisthisfile)+'.png',dpi=300)